In [ ]:
from numpy import exp
from numpy.fft import fft2,  fftfreq, fftshift, ifftshift, rfftfreq
from SFS.src.py.utils import *

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150

In [ ]:
number = 4
folder = "data/SETD_paper/{number}/".format(number = number)
time = get_time(folder + "1/")
n = count_files(folder)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,5))
av = 500

rr = []
Cs = []
for i in range(n)[::1]: 
    q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=i+1)
    assert av <= np.shape(Cq)[0]
    Cq = np.mean(np.abs(Cq[-av:None, :]), axis=0)
    
    r = con["r"]
    rr.append(r)
    color = cm.viridis(i/(n-1))
    title = "$r = {r:.3f}$".format(r=r)
    
    ax[0].loglog(q0[1:], Cq[1:, 0, 0], lw=1., ls="-", color=color, zorder=0)
    x = Cq[1, 0, 0]
    Cs.append(x)
    p = ax[1].plot(r, x, 'x', color=color)


q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=1)
u = con["u"]
ax[0].plot(q0[1:], T*q0[1:]**(-2), 'k--', label="$\\frac{T}{q^2}$")
ax[0].plot(q0[1:], 1.45*T*q0[1:]**(-1.75), 'k--', label="$\\frac{T}{q^\\nu}$")

title = "$u = {u}, N = {N}, L = {L}, T={T}, t={t:1.0e} $".format(u=u,N=N,L=L, T=T,t=time[-1])

fig.colorbar(cm.ScalarMappable(norm=colors.Normalize(vmin=np.min(rr), vmax=np.max(rr)), cmap=cm.viridis), ax=ax, label="$r$")
fig.suptitle(title, y=1.05)
ax[0].legend()
ax[0].set_ylabel("$C(q)$")
ax[0].set_xlabel("$q$")
ax[1].set_ylabel("$C(0)$")
ax[1].set_xlabel("$a$")

plt.show()

# Correlation Length

In [ ]:
from numpy import log as ln
from scipy.optimize import curve_fit

fig1, ax0 = plt.subplots(figsize=(6,4))
fig2, ax1 = plt.subplots(figsize=(6,4))
ax = [ax0, ax1]

xi = []
rs = []
As = []
Bs = []

ax2 = ax[1].twinx()
nn = 1
av = 750
s, e = 1, 40
for j in range(0, n, nn):

    q0, Cq, con, T, L, N = get_Cq_saved(folder, seed=j+1)
    assert av <= np.shape(Cq)[0]
    Cq = np.mean(np.abs(Cq[-av:None, :]), axis=0)[:, 0, 0]
    Cq, q0 = Cq[(Cq!=0)], q0[(Cq!=0)]
    Cq, q0 = Cq[s:e], q0[s:e]
    r = con["r"]


    f = lambda q, A, B, xi : A / (1 + (xi * q)**B )
    lnf = lambda q, A, B, xi : np.log(A / (1 + (q * xi)**B))
    
    XI = 20
    p0 = (T*XI**2, 2, XI)
    
    color = cm.viridis(j/(n-1)) 
    ax[0].loglog(q0, Cq, 'o', ms=3, color=color)
    ax[0].loglog(q0, f(q0, *p0), color='black', lw=1.)
    try: 
        # p = curve_fit(f, q0, Cq, p0=p0)
        p = curve_fit(lnf, q0, np.log(Cq),p0=p0)

        rs.append(r)
        xi.append(p[0][2])
        As.append(p[0][0])
        Bs.append(p[0][1])
        ax[0].loglog(q0, f(q0, *p[0]), color=color, lw=1.)
        ax2.plot(r, xi[-1], '.', color=color)

    except: 
        print("Could not fit {i}".format(i=j))
        continue

ax[1].plot(rs, As, 'r-', marker="o", lw=2, label="$C_0$")
ax2.plot(rs, xi, 'k--', marker="o", lw=2, label="$\\xi$")

title = "$u = {u}, N = {N}, L = {L}, T={T}, t={t:1.0e} $".format(u=u,N=N,L=L, T=T,t=time[-1])
print(title)

ax[0].set_ylabel("$C(q)$")
ax[0].set_xlabel("$q$")
ax[1].set_xlabel("$r$")
ax[1].set_ylabel("$C(0)$")
ax[1].legend(loc=2)
ax2.set_ylabel("$\\xi$")
ax2.legend(loc=1)

fig1.colorbar(cm.ScalarMappable(norm=colors.Normalize(vmin=np.min(rs), vmax=np.max(rs)), cmap=cm.viridis), ax=ax[0], label="$r$",)
fig2.savefig("fig/SETD_paper_fig2.pdf", bbox_inches="tight")

rs, xi, As, Bs = rs[::-1], xi[::-1], As[::-1], Bs[::-1]
rs, xi, As, Bs = np.array(rs), np.array(xi), np.array(As), np.array(Bs)

In [ ]:
xi_c = np.argmax(xi)
B_c = np.argmin(Bs)
A_c = np.argmax(As)

ic = xi_c + 7
rc  = rs[ic]
print(rc)

nn = 0
mm = ic - 6

fig, ax = plt.subplots(1, 1, figsize=(7,6)) 
fig.suptitle("Measure of $C(q; r) = A(r) [1 + (\\xi(r) q)^{B(r)}]^{-1}$")
ax.plot(rs, Bs, ".-", color="g", label="$B(r)$", lw=1)

ax.plot(rs[xi_c], Bs[xi_c], "xr", label="$\\xi_c$", ms=10)
ax.plot(rs[A_c], Bs[A_c], "xy", label="$A_c$", ms=10)
ax.plot(rs[B_c], Bs[B_c], "xb", label="$B_c$", ms=10)
ax.plot(rs[ic], Bs[ic], "v", color="gray", label="$r_c$", ms=10)

ax.plot([rs[mm], rs[mm]], [np.min(Bs), np.max(Bs)], color="tab:blue")
ax.plot([rs[nn], rs[nn]], [np.min(Bs), np.max(Bs)], color="tab:blue")

ax.legend()
plt.show()


fig, ax = plt.subplots(1, 1, figsize=(7,6)) 
ax2 = ax.twinx()
ax2.plot(rs, xi, 'r-', marker=".", lw=.5, label="$\\xi$")
ax.plot(rs, As, 'k-', marker=".", lw=.5, label = "$A$")

ax2.plot(rs[xi_c], xi[xi_c], "xr", label="$\\xi_c$", ms=10, zorder=10)
ax2.plot(rs[A_c], xi[A_c], "xy", label="$A_c$", ms=10)
ax2.plot(rs[B_c], xi[B_c], "xb", label="$B_c$", ms=10, zorder=10)
ax2.plot(rs[ic], xi[ic], "v", color="gray", label="$r_c$", ms=10, zorder=10)

ax.plot([rs[mm], rs[mm]], [np.min(As), np.max(As)], color="tab:blue")
ax.plot([rs[nn], rs[nn]], [np.min(As), np.max(As)], color="tab:blue")

lines2, labels2 = ax.get_legend_handles_labels()
lines1, labels1 = ax2.get_legend_handles_labels()

ax.legend(lines1+lines2, labels1+labels2, loc=1)

plt.show()

In [ ]:
# Plot all
pa      = 0
alpha   = 0.7
ms      = 6
f       = lambda X, a, b : a * X + b

def plot_xy(ax1, ax2, x, y, rc=None, pa=pa, mm=mm+1, nn=nn, color='k', sgn=1):
    if mm==0: mm = None
    xx, yy = x[nn:mm], y[nn:mm]
    p = curve_fit(f, xx, yy)[0]
    label = "${p1:.2f}x + c$".format(p1=p[0],p2=p[1])

    if pa: ax1.plot(x, y, 'o', ms=ms, color="orange")
    ax1.plot(xx, f(xx, *p), 'k--', lw=5, label=label, alpha=alpha, zorder=0)    
    ax1.plot(xx, yy, 'o', ms=ms, color="tab:blue")

    if  rc is None: rc = 0 
    else: ax2.plot([rc, rc], [0, np.max(exp(y))], '--', color=color)
    if pa: ax2.plot(sgn*exp(x) + rc, exp(y), '.', color='orange')

    ax2.plot(sgn*exp(xx) + rc, exp(f(xx, *p)), '--', color=color)
    ax2.plot(sgn*exp(xx) + rc , exp(yy), '.', color="tab:blue")

In [ ]:
fig0, ax0 = plt.subplots(1, 3, figsize=(16, 4))
fig1, ax1 = plt.subplots(1, 3, figsize=(16, 4))
fig2, ax2 = plt.subplots(1, 3, figsize=(16, 4))

ic = xi_c + 7
rc = rs[ic]
mm = xi_c + 1
nn = 0

lnr = ln(np.abs(rs-rc)[:ic])
lnxi = ln(xi[:ic])
lnA = ln(As[:ic])

plot_xy(ax0[0], ax2[0], lnr, lnxi, rc=rc, mm=mm, nn=nn)
plot_xy(ax0[1], ax2[1], lnr, lnA, rc=rc, mm=mm, nn=nn)
plot_xy(ax0[2], ax2[2], lnxi, lnA, mm=mm, nn=nn)

ic = xi_c + 2
rc  = rs[ic]
nn = 2

lnr = ln(np.abs(rs-rc)[ic+1:])
lnxi = ln(xi[ic+1:])
lnA = ln(As[ic+1:])

plot_xy(ax1[0], ax2[0], lnr, lnxi, rc=rc, nn=nn, color="r", sgn=-1)
plot_xy(ax1[1], ax2[1], lnr, lnA, rc=rc, nn=nn, color="r", sgn=-1)
plot_xy(ax1[2], ax2[2], lnxi, lnA, nn=nn, color="r")

ax0[0].legend()
ax0[0].set_xlabel("$\\ln |r - r_c|$")
ax0[0].set_ylabel("$\\ln \\xi$")
ax0[1].legend()
ax0[1].set_xlabel("$\\ln |r - r_c|$")
ax0[1].set_ylabel("$\\ln A_0$")
ax0[2].legend()
ax0[2].set_xlabel("$\\ln \\xi$")
ax0[2].set_ylabel("$\\ln A_0$")

ax1[0].legend()
ax1[1].legend()
ax1[2].legend()
